In [4]:
#Writing A Stacking Agreggator with Scikit-Learn

In [6]:
from __future__ import division

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing

cali_housing = fetch_california_housing()

X = cali_housing.data
y = cali_housing.target

bins = np.arange(6)
 

from sklearn.model_selection import train_test_split

binned_y = np.digitize(y, bins)

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
 
from sklearn.model_selection import GridSearchCV

X_train_prin, X_test_prin, y_train_prin, y_test_prin = train_test_split(X, y,test_size=0.2,stratify=binned_y)

binned_y_train_prin = np.digitize(y_train_prin, bins)
X_1, X_stack, y_1, y_stack = train_test_split(X_train_prin, y_train_prin,test_size=0.33,stratify=binned_y_train_prin )

In [10]:
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV

# 定义参数分布
param_dist = {
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0],
    'oob_score': [True, False],
    'estimator__n_neighbors': [3, 5],  # 使用 'estimator__' 替代 'base_estimator__'
    'n_estimators': [100]
}

# 创建基学习器和 BaggingRegressor
single_estimator = KNeighborsRegressor()
ensemble_estimator = BaggingRegressor(estimator=single_estimator, bootstrap=True)

# 配置 RandomizedSearchCV
pre_gs_inst_bag = RandomizedSearchCV(
    ensemble_estimator,
    param_distributions=param_dist,
    cv=3,
    n_iter=5,
    n_jobs=-1
)

# 拟合模型
pre_gs_inst_bag.fit(X_1, y_1)


RandomizedSearchCV(cv=3,
                   estimator=BaggingRegressor(estimator=KNeighborsRegressor()),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'estimator__n_neighbors': [3, 5],
                                        'max_features': [0.5, 1.0],
                                        'max_samples': [0.5, 1.0],
                                        'n_estimators': [100],
                                        'oob_score': [True, False]})

In [16]:
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor

# 配置 BaggingRegressor
rs_bag = BaggingRegressor(
    estimator=KNeighborsRegressor(n_neighbors=3),  # 使用 'estimator' 替代 'base_estimator'
    max_features=0.5,
    max_samples=0.5,
    n_estimators=3000,
    oob_score=False
)

# 拟合模型
rs_bag.fit(X_1, y_1)


BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=3), max_features=0.5,
                 max_samples=0.5, n_estimators=3000)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

# 定义参数分布
param_dist = {
    'max_features': ['log2', 0.4, 0.5, 0.6, 1.0],
    'max_depth': [2, 3, 4, 5, 6, 7, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 10],
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.05, 0.1, 0.25, 0.275, 0.3, 0.325],
    'loss': ['squared_error', 'huber']  # 将 'ls' 替换为 'squared_error'
}

# 配置 RandomizedSearchCV
pre_gs_inst = RandomizedSearchCV(
    GradientBoostingRegressor(warm_start=True),  # 可以考虑去掉 warm_start=True
    param_distributions=param_dist,
    cv=3,
    n_iter=30,
    n_jobs=-1
)

# 拟合模型
pre_gs_inst.fit(X_1, y_1)


RandomizedSearchCV(cv=3, estimator=GradientBoostingRegressor(warm_start=True),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.25,
                                                          0.275, 0.3, 0.325],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [2, 3, 4, 5, 6, 7, 10],
                                        'max_features': ['log2', 0.4, 0.5, 0.6,
                                                         1.0],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 10],
                                        'n_estimators': [50, 100]})

In [26]:
gbt_inst = GradientBoostingRegressor(**{'learning_rate': 0.05,
 'loss': 'huber',
 'max_depth': 10,
 'max_features': 0.4,
 'min_samples_leaf': 5,
 'n_estimators': 3000,
 'warm_start': True}).fit(X_1, y_1)

In [28]:
y_pred_bag = rs_bag.predict(X_stack)
y_pred_gbt = gbt_inst.predict(X_stack)

In [30]:
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# 计算 R-squared, MAE 和 MAPE
print("R-squared:", r2_score(y_stack, y_pred_bag))
print("MAE:", mean_absolute_error(y_stack, y_pred_bag))

# 检查 y_stack 是否包含零值，避免除以零的错误
if np.any(y_stack == 0):
    print("MAPE: Undefined (y_stack contains zero values)")
else:
    mape = (np.abs(y_stack - y_pred_bag) / y_stack).mean()
    print("MAPE:", mape)


R-squared: 0.5469040932107849
MAE: 0.596173535506006
MAPE: 0.3919971141439516


In [32]:
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# 计算 R-squared, MAE 和 MAPE
print("R-squared:", r2_score(y_stack, y_pred_gbt))
print("MAE:", mean_absolute_error(y_stack, y_pred_gbt))

# 检查 y_stack 是否包含零值，避免除以零的错误
if np.any(y_stack == 0):
    print("MAPE: Undefined (y_stack contains zero values)")
else:
    mape = (np.abs(y_stack - y_pred_gbt) / y_stack).mean()
    print("MAPE:", mape)


R-squared: 0.8436086694196543
MAE: 0.292879792873197
MAPE: 0.1634501006863


In [34]:
y_pred_bag = pre_gs_inst_bag.predict(X_stack)
y_pred_gbt = gbt_inst.predict(X_stack)

preds_df = pd.DataFrame(columns = ['bag', 'gbt'])

preds_df['bag'] = y_pred_bag
preds_df['gbt'] = y_pred_gbt


preds_df

,bag,gbt
0,2.077327,2.255285
1,3.393926,4.402171
2,1.841007,1.543169
3,1.409890,0.609561
4,1.862683,2.164236
...,...,...
5444,1.794637,2.351055
5445,2.760005,4.201641
5446,2.306377,2.428094
5447,1.803343,1.489799


In [36]:
preds_df.corr()

,bag,gbt
bag,1.000000,0.850833
gbt,0.850833,1.000000


In [38]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV

param_dist = {'max_features' : ['sqrt','log2',1.0],
 'min_samples_leaf' : [1, 2, 3, 7, 11],
 'n_estimators': [50, 100],
 'oob_score': [True, False]}

pre_gs_inst = RandomizedSearchCV(ExtraTreesRegressor(warm_start=True,bootstrap=True),
 param_distributions = param_dist,
 cv=3,
 n_iter = 15)

pre_gs_inst.fit(preds_df.values, y_stack)

RandomizedSearchCV(cv=3,
                   estimator=ExtraTreesRegressor(bootstrap=True,
                                                 warm_start=True),
                   n_iter=15,
                   param_distributions={'max_features': ['sqrt', 'log2', 1.0],
                                        'min_samples_leaf': [1, 2, 3, 7, 11],
                                        'n_estimators': [50, 100],
                                        'oob_score': [True, False]})

In [40]:
import copy
 
param_dict = copy.deepcopy(pre_gs_inst.best_params_)
 
param_dict['n_estimators'] = 2000
param_dict['warm_start'] = True
param_dict['bootstrap'] = True
param_dict['n_jobs'] = -1
 
param_dict

{'oob_score': False,
 'n_estimators': 2000,
 'min_samples_leaf': 11,
 'max_features': 1.0,
 'warm_start': True,
 'bootstrap': True,
 'n_jobs': -1}

In [42]:
final_etr = ExtraTreesRegressor(**param_dict)
final_etr.fit(preds_df.values, y_stack)

ExtraTreesRegressor(bootstrap=True, min_samples_leaf=11, n_estimators=2000,
                    n_jobs=-1, warm_start=True)

In [44]:
def handle_X_set(X_train_set):
    y_pred_bag = rs_bag.predict(X_train_set)
    y_pred_gbt = gbt_inst.predict(X_train_set)
    preds_df = pd.DataFrame(columns = ['bag', 'gbt'])

    preds_df['bag'] = y_pred_bag
    preds_df['gbt'] = y_pred_gbt
 
    return preds_df.values

def predict_from_X_set(X_train_set):
    return final_etr.predict(handle_X_set(X_train_set)) 

y_pred = predict_from_X_set(X_test_prin)

In [48]:
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# 计算 R-squared, MAE 和 MAPE
print("R-squared:", r2_score(y_test_prin, y_pred))
print("MAE:", mean_absolute_error(y_test_prin, y_pred))

# 检查 y_test_prin 是否包含零值，避免除以零的错误
if np.any(y_test_prin == 0):
    print("MAPE: Undefined (y_test_prin contains zero values)")
else:
    mape = (np.abs(y_test_prin - y_pred) / y_test_prin).mean()
    print("MAPE:", mape)


R-squared: 0.8491595125811714
MAE: 0.29214802114347904
MAPE: 0.16459182857747895
